## Calculate surface-forced watermass transformation
Presently, information about the products, e.g. names of variables is hard-coded. Could improve robustness by specifying generic names.

In [1]:
%load_ext autoreload
%autoreload 2
from dask.diagnostics import ProgressBar
import so_decadal_variability as so
import numpy as np
import xarray as xr
import os
import gsw
from xhistogram.xarray import histogram
import pandas as pd

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:42823 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 56 Memory: 66.80 GB


In [9]:
fluxname='merra2'
oceanname='en4'
onoceangrid=False
density = 'sigma0'
timeslice = slice('1980','2018')
dg = 0.1
density_edges = np.arange(1024,1028.5,dg)
save = False
savesequential = True
# Load data
ds = so.process.loaddata(fluxname,oceanname,timeslice,onoceangrid)
# Get grid
ds, xgrid = so.grid.get_xgcm(ds,gridlon='longitude',gridlat='latitude')
# Calculate a density variable
ds['sigma0'] = gsw.sigma0(ds['sa'].isel(depth=0).squeeze(),ds['sst'])
ds['b'] = xr.ones_like(ds['sigma0'])
ds['vol4d'] = ds['cell_area']*ds['dz']
ds = ds.rename({'longitude':'lon','latitude':'lat'})
ds = ds.isel(depth=0).squeeze().drop(['depth'])
ds = ds.chunk(chunks={'time':12})
# Calculate WMT
G = so.wmt.calc_watermasstransformation(ds,xgrid,density_edges,density=density,b_ones=False,penetrative_sw=False)

# Save
years = pd.to_datetime(G['time'].values).year.unique()
if save:
    if onoceangrid:
        localdir = '../data/processed/'+fluxname+'_'+oceanname+'_onoceangrid'
    else:
        localdir = '../data/processed/'+fluxname+'_'+oceanname
        
    if ~os.path.isdir(localdir):
        !mkdir $localdir
        
    if savesequential:
        for year in years:
            temp = G.sel(time=str(year))
            filename = 'G.'+density+'.'+str(dg)+'.'+str(year)
            print(filename)
            temp.to_netcdf(localdir+'/'+filename+'.nc')
    else:
        filename = 'G.'+density+'.'+str(dg)
        G.to_netcdf(localdir+'/'+filename+'.nc')

/home/aos/graemem/miniconda3/envs/so_decadal_variability/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


KeyError: 'ht'

In [7]:
so.process._get_fluxpath(fluxname)

'/local/projects/so_decadal_variability/flux/SO_flux_*jra55_1979-2019.nc'